In [100]:
import os
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import simfin as sf
from simfin.names import *

In [101]:
load_dotenv()

api_key = os.getenv("API_KEY")

print(api_key[:5])  


9104e


In [102]:

# Set SimFin data directory
sf.set_api_key(api_key)

sf.set_data_dir('/Users/ayushsingh/Desktop/MBD/Python 2/Group Assignment/simfin_data/')

# Load datasets
df_companies = sf.load_companies(market='us')
df_share_prices = sf.load_shareprices(market='us', variant='daily')

Dataset "us-companies" on disk (13 days old).
- Loading from disk ... Done!
Dataset "us-shareprices-daily" on disk (13 days old).
- Loading from disk ... 

/opt/anaconda3/lib/python3.12/site-packages/simfin/load.py:154: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv(path, sep=';', header=0,
/opt/anaconda3/lib/python3.12/site-packages/simfin/load.py:154: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv(path, sep=';', header=0,


Done!


In [112]:
import pandas as pd
import numpy as np

class ETL:
    def __init__(self, share_prices_df, companies_df, tickers, output_file='filtered_5_companies_stock_data.csv'):
        self.df_share_prices = share_prices_df.copy()
        self.df_companies = companies_df.copy()
        self.tickers = tickers
        self.output_file = output_file
        self.df_cleaned = None

        # Final feature set
        self.final_features = [
            'ticker', 'date',
            'open', 'high', 'low', 'close', 'adj. close', 'volume',
            'daily_return', 'volatility',
            '5_day_ma', '10_day_ma',
            'trend', 'next_day_close'
        ]

    def reset_and_clean_columns(self):
        self.df_share_prices.reset_index(inplace=True)
        self.df_companies.reset_index(inplace=True)
        self.df_share_prices.columns = self.df_share_prices.columns.str.lower()
        self.df_companies.columns = self.df_companies.columns.str.lower()

    def handle_dates_and_duplicates(self):
        self.df_share_prices['date'] = pd.to_datetime(self.df_share_prices['date'], errors='coerce')
        self.df_companies.drop_duplicates(inplace=True)
        self.df_share_prices.drop_duplicates(inplace=True)

    def handle_nulls_and_missing(self):
        self.df_companies.ffill(inplace=True)
        self.df_share_prices.ffill(inplace=True)
        self.df_share_prices.dropna(subset=['close'], inplace=True)

    def feature_engineering(self):
        group = self.df_share_prices.groupby('ticker')

        # Daily return
        self.df_share_prices['daily_return'] = group['close'].pct_change()

        # Volatility (7-day rolling std)
        self.df_share_prices['volatility'] = group['close'].transform(lambda x: x.rolling(7).std())

        # Moving averages
        self.df_share_prices['5_day_ma'] = group['close'].transform(lambda x: x.rolling(5).mean())
        self.df_share_prices['10_day_ma'] = group['close'].transform(lambda x: x.rolling(10).mean())

        # Trend variable (classification target)
        self.df_share_prices['trend'] = (self.df_share_prices['daily_return'] > 0).astype(int)

        # Price prediction target
        self.df_share_prices['next_day_close'] = group['close'].shift(-1)

    def filter_data(self):
        # Filter for selected tickers and last 3 years
        three_years_ago = pd.Timestamp.today() - pd.DateOffset(years=3)
        self.df_share_prices = self.df_share_prices[
            (self.df_share_prices['ticker'].isin(self.tickers)) &
            (self.df_share_prices['date'] >= three_years_ago)
        ]
        self.df_companies = self.df_companies[self.df_companies['ticker'].isin(self.tickers)]

    def merge_and_save(self):
        df_merged = self.df_share_prices.merge(self.df_companies, on='ticker', how='left')
        model_columns = [col for col in self.final_features if col not in ['ticker', 'date']]
        df_filtered = df_merged.dropna(subset=model_columns)

        # Optional: Keep only tickers with enough data
        df_filtered = df_filtered.groupby('ticker').filter(lambda x: len(x) >= 60)

        self.df_cleaned = df_filtered[self.final_features]
        self.df_cleaned.to_csv(self.output_file, index=False)

    def run_pipeline(self):
        self.reset_and_clean_columns()
        self.handle_dates_and_duplicates()
        self.handle_nulls_and_missing()
        self.feature_engineering()
        self.filter_data()
        self.merge_and_save()
        return self.df_cleaned

In [113]:
# Assuming you already have df_share_prices and df_companies loaded
selected_tickers = ["AAPL", "WMT", "GOOG", "NFLX", "MSFT"]

etl = ETL(share_prices_df=df_share_prices, companies_df=df_companies, tickers=selected_tickers)
df_cleaned = etl.run_pipeline()


In [114]:
df_cleaned['ticker'].unique()

array(['AAPL', 'GOOG', 'MSFT', 'NFLX', 'WMT'], dtype=object)

SimFinId   Open   High    Low  Close  Adj. Close   Volume  \
Ticker Date                                                                    
A      2019-04-11     45846  81.88  81.92  80.89  81.08       77.87  1071479   
       2019-04-12     45846  81.43  82.06  80.90  80.98       77.77  1249295   
       2019-04-15     45846  81.00  81.13  79.91  80.40       77.22  1627268   
       2019-04-16     45846  80.82  80.96  77.19  77.55       74.48  3441597   
       2019-04-17     45846  78.15  78.32  74.46  75.43       72.44  4471971   
...                     ...    ...    ...    ...    ...         ...      ...   
ZYXI   2024-03-08    171401  12.78  13.19  12.74  12.86       12.86   213848   
       2024-03-11    171401  12.83  13.07  12.67  12.69       12.69   150265   
       2024-03-12    171401  12.69  12.87  12.59  12.60       12.60   151053   
       2024-03-13    171401  12.57  12.81  12.57  12.69       12.69    89921   
       2024-03-14    171401  12.69  12.73  12.18  12.19       12.19   313909   

                   Dividend  Shares Outstanding  
Ticker Date                                      
A      2019-04-11       NaN         317515869.0  
       2019-04-12       NaN         317515869.0  
       2019-04-15       NaN         317515869.0  
       2019-04-16       NaN         317515869.0  
       2019-04-17       NaN         317515869.0  
...                     ...                 ...  
ZYXI   2024-03-08       NaN          32170182.0  
       2024-03-11       NaN          32170182.0  
       2024-03-12       NaN          32170182.0  
       2024-03-13       NaN          32170182.0  
       2024-03-14       NaN          32170182.0  

[5815264 rows x 9 columns]

In [47]:
import pandas as pd

df_share_prices.reset_index()

# Count number of rows per ticker
ticker_counts = df_share_prices['Ticker'].value_counts().reset_index()
ticker_counts.columns = ['icker', 'row_count']

# Sort in descending order (most data first)
ticker_counts = ticker_counts.sort_values(by='row_count', ascending=False)

print("📊 Companies with most data:")
print(ticker_counts)


KeyError: 'Ticker'

In [97]:
# Resetting the index and converting columns to lower case

df_share_prices.reset_index(inplace=True)
df_companies.reset_index(inplace=True)
df_share_prices.columns = df_share_prices.columns.str.lower()
df_companies.columns = df_companies.columns.str.lower()



ValueError: cannot insert level_0, already exists

In [53]:
ticker_counts = df_share_prices['ticker'].value_counts().reset_index()
ticker_counts.columns = ['icker', 'row_count']

# Sort in descending order (most data first)
ticker_counts = ticker_counts.sort_values(by='row_count', ascending=False)

print("📊 Companies with most data:")
print(ticker_counts)

# List of tickers you're interested in
top_tickers = ['BRN', 'DBVT', 'DBX', 'BWA']

# Ensure columns are lowercase (if not already)
df_companies.columns = df_companies.columns.str.lower()

# Filter for those tickers
top_company_names = df_companies[df_companies['ticker'].isin(top_tickers)][['ticker', 'company name']]

print("🏢 Full Company Names:")
print(top_company_names)




📊 Companies with most data:
     icker  row_count
0        A       1240
2198   BRN       1240
2188  DBVT       1240
2189   DBX       1240
2190   BWA       1240
...    ...        ...
5582  TCDA          1
5583   RDC          1
5584  CLVS          1
5585   PNT          1
5586  AMEH          1

[5587 rows x 2 columns]
🏢 Full Company Names:
     ticker               company name
861     BRN  Barnwell Industries, Inc.
912     BWA             BORGWARNER INC
1552   DBVT      DBV Technologies S.A.
1553    DBX              DROPBOX, INC.


In [98]:

df_share_prices['date'] = pd.to_datetime(df_share_prices['date'], errors='coerce')

# Drop duplicate rows
df_companies.drop_duplicates(inplace=True)
df_share_prices.drop_duplicates(inplace=True)

# Handle missing values
df_companies.fillna(method='ffill', inplace=True)  # Forward fill missing company data
df_share_prices.fillna(method='ffill', inplace=True)  # Forward fill stock prices

# Remove rows where essential data is missing
df_share_prices.dropna(subset=['close'], inplace=True)


KeyError: 'date'

In [99]:
Q1 = df_share_prices['close'].quantile(0.25)
Q3 = df_share_prices['close'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
df_share_prices = df_share_prices[(df_share_prices['close'] >= lower_bound) & (df_share_prices['close'] <= upper_bound)]

# Feature Engineering - Create lag and moving average features
df_share_prices['prev_close'] = df_share_prices.groupby('ticker')['close'].shift(1)
df_share_prices['ma7'] = df_share_prices.groupby('ticker')['close'].transform(lambda x: x.rolling(7).mean())
df_share_prices['ma30'] = df_share_prices.groupby('ticker')['close'].transform(lambda x: x.rolling(30).mean())

# Merge company details with share prices
df_cleaned = df_share_prices.merge(df_companies, on='ticker', how='left')

df_cleaned.to_csv('cleaned_stock_data.csv', index=False)
print("✅ Data saved as 'cleaned_stock_data.csv' instead of Excel due to size limitations.")


KeyError: 'close'

In [39]:
df_cleaned

,ticker,date,simfinid_x,open,high,low,close,adj. close,volume,dividend,...,simfinid_y,company name,industryid,isin,end of financial year (month),number employees,business summary,market,cik,main currency
0,A,2019-04-11,45846,81.88,81.92,80.89,81.08,77.87,1071479,NaN,...,45846,AGILENT TECHNOLOGIES INC,106001.0,US00846U1016,10.0,16400.0,Agilent Technologies Inc is engaged in life sc...,us,1090872.0,USD
1,A,2019-04-12,45846,81.43,82.06,80.90,80.98,77.77,1249295,NaN,...,45846,AGILENT TECHNOLOGIES INC,106001.0,US00846U1016,10.0,16400.0,Agilent Technologies Inc is engaged in life sc...,us,1090872.0,USD
2,A,2019-04-15,45846,81.00,81.13,79.91,80.40,77.22,1627268,NaN,...,45846,AGILENT TECHNOLOGIES INC,106001.0,US00846U1016,10.0,16400.0,Agilent Technologies Inc is engaged in life sc...,us,1090872.0,USD
3,A,2019-04-16,45846,80.82,80.96,77.19,77.55,74.48,3441597,NaN,...,45846,AGILENT TECHNOLOGIES INC,106001.0,US00846U1016,10.0,16400.0,Agilent Technologies Inc is engaged in life sc...,us,1090872.0,USD
4,A,2019-04-17,45846,78.15,78.32,74.46,75.43,72.44,4471971,NaN,...,45846,AGILENT TECHNOLOGIES INC,106001.0,US00846U1016,10.0,16400.0,Agilent Technologies Inc is engaged in life sc...,us,1090872.0,USD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5023036,ZYXI,2024-03-14,171401,12.69,12.73,12.18,12.19,12.19,313909,0.1,...,7962794,"BurgerFi International, Inc.",106004.0,US98986M1036,12.0,768.0,"Zynex, Inc. engages in the design, manufacture...",us,1705873.0,USD
5023037,ZYXI,2024-03-14,171401,12.69,12.73,12.18,12.19,12.19,313909,0.1,...,689587,"BurgerFi International, Inc.",106004.0,US98986M1036,12.0,768.0,"Zynex, Inc. engages in the design, manufacture...",us,1705873.0,USD
5023038,ZYXI,2024-03-14,171401,12.69,12.73,12.18,12.19,12.19,313909,0.1,...,6480955,"BurgerFi International, Inc.",106004.0,US98986M1036,12.0,768.0,"Zynex, Inc. engages in the design, manufacture...",us,1906324.0,USD
5023039,ZYXI,2024-03-14,171401,12.69,12.73,12.18,12.19,12.19,313909,0.1,...,6481013,"BurgerFi International, Inc.",106004.0,US98986M1036,12.0,768.0,"Zynex, Inc. engages in the design, manufacture...",us,1130464.0,USD


In [12]:
df_cleaned.columns

Index(['ticker', 'date', 'simfinid_x', 'open', 'high', 'low', 'close',
       'adj. close', 'volume', 'dividend', 'shares outstanding', 'prev_close',
       'ma7', 'ma30', 'simfinid_y', 'company name', 'industryid', 'isin',
       'end of financial year (month)', 'number employees', 'business summary',
       'market', 'cik', 'main currency'],
      dtype='object')

In [4]:
import pandas as pd

# Load the cleaned stock data
df = pd.read_csv("cleaned_stock_data.csv")

# Define the tickers to keep
selected_tickers = ["AAPL", "WMT", "GOOG", "TSLA", "NVDA"]

# Filter the dataset for only these tickers
df_filtered = df[df["ticker"].isin(selected_tickers)]

# Save the filtered dataset to a new CSV file
df_filtered.to_csv("filtered_5_companies_stock_data.csv", index=False)

print("✅ Filtered dataset saved as 'filtered_stock_data.csv'")


✅ Filtered dataset saved as 'filtered_stock_data.csv'


In [38]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import joblib

# Load cleaned stock data
df = pd.read_csv('cleaned_stock_data.csv')

# 🔹 User Input: Select a Company
company_ticker = input("Enter the company ticker (e.g., GOOGL, AAPL, MSFT): ").strip().upper()

# 🔹 Check if the company exists
if company_ticker not in df['ticker'].unique():
    print(f"❌ Error: Company '{company_ticker}' not found in dataset.")
    exit()

# Filter for selected company
df_company = df[df['ticker'] == company_ticker].copy()

# Convert 'date' to datetime and sort by date
df_company['date'] = pd.to_datetime(df_company['date'])
df_company = df_company.sort_values(by='date')

# 🔹 Create Target Variable: Trend Prediction (1 = Up, 0 = Down)
df_company['trend'] = (df_company['close'].shift(-1) > df_company['close']).astype(int)

# Drop last row (no future data)
df_company = df_company[:-1]

# 🔹 Feature Engineering
df_company['daily_return'] = df_company['close'].pct_change()
df_company['volatility'] = df_company['close'].rolling(7).std()
df_company['ma7'] = df_company['close'].rolling(7).mean()
df_company['ma30'] = df_company['close'].rolling(30).mean()
df_company.dropna(inplace=True)  # Remove NaN values

# 🔹 Prepare Training Data
features = ['close', 'daily_return', 'volatility', 'ma7', 'ma30']
X = df_company[features]
y = df_company['trend']

# Split into training & testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 🔹 Hyperparameter Tuning with GridSearchCV
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rf = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Get Best Model
best_model = grid_search.best_estimator_
print(f"✅ Best Parameters: {grid_search.best_params_}")

# Evaluate Model
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"✅ Model Accuracy: {accuracy:.2f}")
print(classification_report(y_test, y_pred))

# 🔹 Predict Next Day's Trend
latest_data = X.iloc[[-1]]  # Get the most recent day’s features
prediction = best_model.predict(latest_data)

# Show Prediction
trend_prediction = "Up 📈" if prediction[0] == 1 else "Down 📉"
print(f"📊 Prediction for {company_ticker} on the next trading day: {trend_prediction}")

# Save Model
joblib.dump(best_model, f'{company_ticker}_trend_model.pkl')
print(f"✅ Model saved as '{company_ticker}_trend_model.pkl'")


✅ Best Parameters: {'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 100}
✅ Model Accuracy: 0.66
              precision    recall  f1-score   support

           0       0.69      0.35      0.47        31
           1       0.66      0.88      0.75        43

    accuracy                           0.66        74
   macro avg       0.67      0.62      0.61        74
weighted avg       0.67      0.66      0.63        74

📊 Prediction for GOOG on the next trading day: Up 📈
✅ Model saved as 'GOOG_trend_model.pkl'
